In [1]:
import numpy as np
import sys
import os
from typing import Sequence
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from gradient_descendent import gradient_descendent
from normalize import MinMaxNormalizer, StandardScaler
from loss_fn_tarefa2 import make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function
from plots.plots_tarefa2 import plot_best_by_loss

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho2dados.xlsx')

df.head()

,x,y,z
0,-5.0,-5.0,-458.963629
1,-5.0,-3.8,-520.361381
2,-5.0,-2.6,-593.039231
3,-5.0,-1.4,-606.776605
4,-5.0,-0.2,-657.401892


## EDA dados

In [3]:
df.describe()

,x,y,z
count,81.000000,81.000000,81.000000
mean,-0.200000,-0.200000,36.990573
std,3.117691,3.117691,294.680404
min,-5.000000,-5.000000,-657.401892
25%,-2.600000,-2.600000,-42.780939
50%,-0.200000,-0.200000,59.403645
75%,2.200000,2.200000,170.599518
max,4.600000,4.600000,614.700100


In [4]:
df.isna().sum()

x    0
y    0
z    0
dtype: int64

In [5]:
df_pivot = df.pivot(index='y', columns='x', values='z')

# Gerar a malha
x_axis = df_pivot.columns.values
y_axis = df_pivot.index.values
z_grid = df_pivot.values

fig = go.Figure()

# Adicionar a Superfície
fig.add_trace(go.Surface(
    x=x_axis,
    y=y_axis,
    z=z_grid,
    colorscale='Viridis',
    opacity=0.9,
    contours_z=dict(show=True, usecolormap=True, project_z=True, highlightcolor="white"),
    name='Superfície dos Dados'
))

# Adicionar os Pontos de Dados Originais
fig.add_trace(go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(size=3, color='red', symbol='circle'),
    name='Pontos de Dados Originais'
))

# Melhorar o Layout
fig.update_layout(
    title=dict(text='z = f(x, y)', x=0.5),
    scene=dict(
        xaxis_title='Eixo X',
        yaxis_title='Eixo Y',
        zaxis_title='Eixo Z (Valor)'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

In [6]:
X = np.column_stack([df['x'] ** 3, df['y'] ** 2, np.ones(len(df['x']))])

true_weights = np.linalg.inv((X.T @ X) + (X.T @ X).T) @ (2 * X.T @ df['z'])
true_weights

array([ 5.0081046 ,  5.73419945, 10.59963756])

## Calcular as funções de perda

### Configurações

In [7]:
def make_function(w):
    def f(x, y):
        if np.isscalar(x) and np.isscalar(y):
            return w @ np.array([x, y, 1])
        else:
            return np.array([w @ np.array([xi, yi, 1]) for xi, yi in zip(x, y)])
    return f

features = df[['x', 'y']]
y = df['z']

# Criar os objetos para Normalização e Padronização
min_max_scaler = MinMaxNormalizer(-1, 1)
standard_scaler = StandardScaler()
min_max_scaler_z = MinMaxNormalizer(-1, 1)
standard_scaler_z = StandardScaler()

# Cria as cópias dos dados para normalização e padronização
cubed_squared_features = features.copy()  # Mantém os dados originais para comparação
cubed_squared_features.loc[:, 'x'] = cubed_squared_features['x'] ** 3
cubed_squared_features.loc[:, 'y'] = cubed_squared_features['y'] ** 2
features_normalized = cubed_squared_features.copy()
features_standardized = cubed_squared_features.copy()

# Ajusta os normalizadores e padronizadores aos dados
min_max_scaler.fit(features_normalized)
standard_scaler.fit(features_standardized)
min_max_scaler_z.fit(y.values.reshape(-1, 1))
standard_scaler_z.fit(y.values.reshape(-1, 1))

# Normaliza e Padroniza os dados
features_normalized = min_max_scaler.normalize(features_normalized)
features_standardized = standard_scaler.normalize(features_standardized)
y_normalized = min_max_scaler_z.normalize(y.values.reshape(-1, 1)).flatten()
y_standardized = standard_scaler_z.normalize(y.values.reshape(-1, 1)).flatten()

# Listas para iteração
features_list = [cubed_squared_features, features_normalized, features_standardized]
features_names = ['Original', 'Normalized', 'Standardized']
#target_list = [y.values, y_normalized, y_standardized]
target_list = [y.values, y.values, y.values]
loss_fn_names = ['MSE', 'MAE', 'RMSE']
loss_fn_makers = [make_mse_loss_function, make_mae_loss_function, make_rmse_loss_function]

learning_rates = [0.001, 0.0001, 0.00001]
initial_weights = [
    np.zeros(3),  # Zero sempre seguro
    np.array([0.1, 0.1, 0.1]),  # Valores pequenos
    np.random.randn(3) * 0.01  # Inicialização normal pequena
]
n_iterations = 10000
tolerance = 1e-6

### Rodar os experimentos

In [8]:
dict_results = {}

for feature_set, feature_name, target in zip(features_list, features_names, target_list):
    x_data = feature_set['x'].values
    y_data = feature_set['y'].values
    z_data = target

    x_data_orig = cubed_squared_features['x'].values
    y_data_orig = cubed_squared_features['y'].values
    z_data_orig = y

    for loss_fn_name, loss_fn_maker in zip(loss_fn_names, loss_fn_makers):
        loss_function, grad_loss_function = None, None

        loss_function, grad_loss_function = loss_fn_maker(x_data, y_data, z_data)

        for lr in learning_rates:
            for initial_w in initial_weights:
                try:
                    weights, losses, n_iters = gradient_descendent(
                        initial_w, loss_function, grad_loss_function,
                        learning_rate=lr, max_iter=n_iterations, tolerance=tolerance, stopping_criteria=[1, 2, 3]
                    )

                    # Verificar se há overflow
                    if np.any(np.isnan(weights[-1])) or np.any(np.isinf(weights[-1])):
                        print(f"Overflow detectado para LR={lr}, Initial_W={initial_w}")
                        continue

                except Exception as e:
                    print(f"Erro para LR={lr}, Initial_W={initial_w}: {e}")
                    continue

                weights_raw = weights[-1].copy()
                if feature_name == 'Standardized':
                    weights[-1] = standard_scaler.desnormalize_weights(weights[-1])
                elif feature_name == 'Normalized':
                    weights[-1] = min_max_scaler.desnormalize_weights(weights[-1])

                function_aprox = make_function(weights[-1])

                mse_final = np.mean((z_data_orig - function_aprox(x_data_orig, y_data_orig)) ** 2)
                rmse_final = np.sqrt(mse_final)
                mae_final = np.mean(np.abs(z_data_orig - function_aprox(x_data_orig, y_data_orig)))

                key = (feature_name, loss_fn_name, lr, tuple(initial_w))
                dict_results[key] = {
                    'weights': weights,
                    'weights_raw': weights_raw,
                    'losses': losses,
                    'n_iters': n_iters,
                    'mse_final': mse_final,
                    'rmse_final': rmse_final,
                    'mae_final': mae_final
                }

df_result = pd.DataFrame([
    {
        'Feature_Set': key[0],
        'Loss_Function': key[1],
        'Learning_Rate': key[2],
        'Initial_Weights': key[3],
        'Weights_raw': value['weights_raw'],
        'Final_Weights': value['weights'][-1],
        'Final_Loss': value['losses'][-1],
        'MSE_Final': value['mse_final'],
        'RMSE_Final': value['rmse_final'],
        'MAE_Final': value['mae_final'],
        'Iterations': value['n_iters']
    }
    for key, value in dict_results.items()
])

df_result_to_save = df_result.copy()
df_result_to_save['Learning_Rate'] = df_result_to_save['Learning_Rate'].apply(lambda x: f"{float(x):.6g}")
df_result_to_save['Initial_Weights'] = df_result_to_save['Initial_Weights'].apply(lambda x: np.array(x).tolist())
df_result_to_save['Final_Weights'] = df_result_to_save['Final_Weights'].apply(lambda x: np.array(x).tolist())

df_result_to_save['Initial_Weights'] = df_result_to_save['Initial_Weights'].apply(lambda x: str(x))
df_result_to_save['Final_Weights'] = df_result_to_save['Final_Weights'].apply(lambda x: str(x))

df_result_to_save.to_excel('../output/tarefa2_results.xlsx', index=False, float_format="%.6g")

df_result.sort_values(by='MSE_Final').groupby('Loss_Function').head(3).reset_index(drop=True)

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\.venv\Lib\site-packages\numpy\_core\_methods.py:134: RuntimeWarning:

overflow encountered in reduce

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:14: RuntimeWarning:

overflow encountered in square

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:63: RuntimeWarning:

invalid value encountered in scalar subtract

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:19: RuntimeWarning:

overflow encountered in matmul

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\utils\loss_fn_tarefa2.py:19: RuntimeWarning:

invalid value encountered in matmul

c:\Users\mathe\Documents\Meus Codes\rna_tarefas_mestrado\libs\gradient_descendent.py:49: RuntimeWarning:

invalid value encountered in subtract



Overflow detectado para LR=0.001, Initial_W=[0. 0. 0.]
Overflow detectado para LR=0.001, Initial_W=[0.1 0.1 0.1]
Overflow detectado para LR=0.001, Initial_W=[-0.0008095   0.0034561   0.01245966]


,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Weights_raw,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Standardized,MSE,0.0010,"(-0.0008095019450254399, 0.0034561029535432643...","[288.0966188051318, 48.82206766173166, 36.9885...","[5.007836663413999, 5.733892693254057, 10.5990...",372.092180,372.092180,19.289691,15.294893,4913
1,Standardized,MSE,0.0010,"(0.0, 0.0, 0.0)","[288.09661884843933, 48.822067476832046, 36.98...","[5.007836664166791, 5.7338926715385785, 10.599...",372.092180,372.092180,19.289691,15.294893,4913
2,Standardized,MSE,0.0010,"(0.1, 0.1, 0.1)","[288.0965933196544, 48.82206760284204, 36.9885...","[5.007836220412937, 5.733892686337784, 10.5990...",372.092181,372.092181,19.289691,15.294892,4912
3,Original,RMSE,0.0010,"(0.1, 0.1, 0.1)","[5.001863826357645, 6.213147485618186, 2.36617...","[5.001863826357645, 6.213147485618186, 2.36617...",20.041726,401.670789,20.041726,15.672183,10000
4,Original,RMSE,0.0010,"(-0.0008095019450254399, 0.0034561029535432643...","[5.001811598900067, 6.217155683909959, 2.29727...","[5.001811598900067, 6.217155683909959, 2.29727...",20.054125,402.167937,20.054125,15.681766,10000
5,Original,RMSE,0.0010,"(0.0, 0.0, 0.0)","[5.001803905449378, 6.217746118086512, 2.28712...","[5.001803905449378, 6.217746118086512, 2.28712...",20.055960,402.241520,20.055960,15.683177,10000
6,Original,MAE,0.0010,"(0.0, 0.0, 0.0)","[5.01816187654278, 6.283904691357363, 1.135543...","[5.01816187654278, 6.283904691357363, 1.135543...",15.825815,412.167477,20.301908,15.825815,4949
7,Original,MAE,0.0010,"(0.1, 0.1, 0.1)","[5.0166004938267905, 6.285862716048745, 1.1116...","[5.0166004938267905, 6.285862716048745, 1.1116...",15.829857,412.194659,20.302578,15.829857,4256
8,Original,MAE,0.0001,"(0.1, 0.1, 0.1)","[5.007548316049207, 6.24645397530912, 0.644371...","[5.007548316049207, 6.24645397530912, 0.644371...",15.919502,416.256517,20.402365,15.919502,9749


## Resultados

### MSE

In [9]:
_ = plot_best_by_loss(df_result, "MSE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado MSE:
Feature Set: Standardized
Learning Rate: 0.001
Final Loss: 372.09217994900183
Pesos: [ 5.00783666  5.73389269 10.59907094]


In [10]:
df_result_mse = df_result[df_result['Loss_Function'] == 'MSE'].sort_values(by='MSE_Final').reset_index(drop=True)
df_result_mse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Weights_raw,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Standardized,MSE,0.00100,"(-0.0008095019450254399, 0.0034561029535432643...","[288.0966188051318, 48.82206766173166, 36.9885...","[5.007836663413999, 5.733892693254057, 10.5990...",372.092180,372.092180,19.289691,15.294893,4913
1,Standardized,MSE,0.00100,"(0.0, 0.0, 0.0)","[288.09661884843933, 48.822067476832046, 36.98...","[5.007836664166791, 5.7338926715385785, 10.599...",372.092180,372.092180,19.289691,15.294893,4913
2,Standardized,MSE,0.00100,"(0.1, 0.1, 0.1)","[288.0965933196544, 48.82206760284204, 36.9885...","[5.007836220412937, 5.733892686337784, 10.5990...",372.092181,372.092181,19.289691,15.294892,4912
3,Original,MSE,0.00010,"(0.1, 0.1, 0.1)","[5.004883393524895, 5.981410684267237, 6.34990...","[5.004883393524895, 5.981410684267237, 6.34990...",379.972200,379.972200,19.492876,15.316278,10000
4,Original,MSE,0.00010,"(-0.0008095019450254399, 0.0034561029535432643...","[5.0048574353025606, 5.983402849064158, 6.3156...","[5.0048574353025606, 5.983402849064158, 6.3156...",380.099719,380.099719,19.496146,15.318211,10000
5,Original,MSE,0.00010,"(0.0, 0.0, 0.0)","[5.004853554896708, 5.983700650964379, 6.31053...","[5.004853554896708, 5.983700650964379, 6.31053...",380.118869,380.118869,19.496637,15.318501,10000
6,Original,MSE,0.00001,"(0.1, 0.1, 0.1)","[5.001058032080917, 6.274988199168381, 1.30309...","[5.001058032080917, 6.274988199168381, 1.30309...",409.802224,409.802224,20.243572,15.820022,10000
7,Original,MSE,0.00001,"(-0.0008095019450254399, 0.0034561029535432643...","[5.00100124698145, 6.279346173866839, 1.228177...","[5.00100124698145, 6.279346173866839, 1.228177...",410.412454,410.412454,20.258639,15.830440,10000
8,Original,MSE,0.00001,"(0.0, 0.0, 0.0)","[5.0009927583711935, 6.279997632595718, 1.2169...","[5.0009927583711935, 6.279997632595718, 1.2169...",410.504096,410.504096,20.260901,15.831998,10000
9,Standardized,MSE,0.00010,"(0.1, 0.1, 0.1)","[249.1416385636257, 42.231830137837406, 31.998...","[4.330702099721204, 4.959904277874321, 9.16498...",1959.165260,1959.165260,44.262459,33.734391,10000


### RMSE

In [11]:
_ = plot_best_by_loss(df_result, "RMSE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado RMSE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 20.041726203384506
Pesos: [5.00186383 6.21314749 2.36617984]


In [12]:
df_result_rmse = df_result[df_result['Loss_Function'] == 'RMSE'].sort_values(by='RMSE_Final').drop(columns='Final_Loss').reset_index(drop=True)
df_result_rmse

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Weights_raw,Final_Weights,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,RMSE,0.00100,"(0.1, 0.1, 0.1)","[5.001863826357645, 6.213147485618186, 2.36617...","[5.001863826357645, 6.213147485618186, 2.36617...",401.670789,20.041726,15.672183,10000
1,Original,RMSE,0.00100,"(-0.0008095019450254399, 0.0034561029535432643...","[5.001811598900067, 6.217155683909959, 2.29727...","[5.001811598900067, 6.217155683909959, 2.29727...",402.167937,20.054125,15.681766,10000
2,Original,RMSE,0.00100,"(0.0, 0.0, 0.0)","[5.001803905449378, 6.217746118086512, 2.28712...","[5.001803905449378, 6.217746118086512, 2.28712...",402.241520,20.055960,15.683177,10000
3,Original,RMSE,0.00010,"(0.1, 0.1, 0.1)","[5.000111977584725, 6.291770872525102, 0.60501...","[5.000111977584725, 6.291770872525102, 0.60501...",415.759530,20.390182,15.922895,10000
4,Original,RMSE,0.00010,"(-0.0008095019450254399, 0.0034561029535432643...","[5.000015371203457, 6.294603347747767, 0.52270...","[5.000015371203457, 6.294603347747767, 0.52270...",416.495231,20.408215,15.935522,10000
5,Original,RMSE,0.00010,"(0.0, 0.0, 0.0)","[5.000004796867698, 6.295228044651212, 0.51059...","[5.000004796867698, 6.295228044651212, 0.51059...",416.602406,20.410840,15.937325,10000
6,Original,RMSE,0.00001,"(0.1, 0.1, 0.1)","[4.4955877486300215, 0.7459156620336218, 0.137...","[4.4955877486300215, 0.7459156620336218, 0.137...",6136.013689,78.332711,63.852982,10000
7,Original,RMSE,0.00001,"(-0.0008095019450254399, 0.0034561029535432643...","[4.451610508518476, 0.6383814056200646, 0.0492...","[4.451610508518476, 0.6383814056200646, 0.0492...",6467.854501,80.422972,65.528344,10000
8,Original,RMSE,0.00001,"(0.0, 0.0, 0.0)","[4.4513994783966515, 0.6352689964662742, 0.036...","[4.4513994783966515, 0.6352689964662742, 0.036...",6475.589258,80.471046,65.569009,10000
9,Standardized,RMSE,0.00100,"(0.1, 0.1, 0.1)","[9.860535839657262, 1.7512469176741832, 1.3501...","[0.17140066795488465, 0.20567465464406204, 0.3...",81282.131892,285.100214,209.298387,10000


### MAE

In [13]:
_ = plot_best_by_loss(df_result, "MAE", df, features_normalized, features_standardized, show_original_points=True)

Melhor resultado MAE:
Feature Set: Original
Learning Rate: 0.001
Final Loss: 15.825814843820108
Pesos: [5.01816188 6.28390469 1.13554321]


In [14]:
df_result_mae = df_result[df_result['Loss_Function'] == 'MAE'].sort_values(by='MAE_Final').drop(columns='Final_Loss').reset_index(drop=True)
df_result_mae

,Feature_Set,Loss_Function,Learning_Rate,Initial_Weights,Weights_raw,Final_Weights,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Original,MAE,0.00100,"(0.0, 0.0, 0.0)","[5.01816187654278, 6.283904691357363, 1.135543...","[5.01816187654278, 6.283904691357363, 1.135543...",412.167477,20.301908,15.825815,4949
1,Original,MAE,0.00100,"(0.1, 0.1, 0.1)","[5.0166004938267905, 6.285862716048745, 1.1116...","[5.0166004938267905, 6.285862716048745, 1.1116...",412.194659,20.302578,15.829857,4256
2,Original,MAE,0.00100,"(-0.0008095019450254399, 0.0034561029535432643...","[5.017439189413042, 6.304686720237419, 0.63715...","[5.017439189413042, 6.304686720237419, 0.63715...",416.377357,20.405327,15.896891,1382
3,Original,MAE,0.00010,"(0.1, 0.1, 0.1)","[5.007548316049207, 6.24645397530912, 0.644371...","[5.007548316049207, 6.24645397530912, 0.644371...",416.256517,20.402365,15.919502,9749
4,Original,MAE,0.00010,"(-0.0008095019450254399, 0.0034561029535432643...","[5.006344927684392, 6.243138769620673, 0.56411...","[5.006344927684392, 6.243138769620673, 0.56411...",417.087088,20.422710,15.934524,9694
5,Original,MAE,0.00010,"(0.0, 0.0, 0.0)","[5.00462019753064, 6.232967753086868, 0.549053...","[5.00462019753064, 6.232967753086868, 0.549053...",417.440771,20.431367,15.945192,9521
6,Original,MAE,0.00001,"(0.1, 0.1, 0.1)","[3.617496393086131, 0.6403988641975492, 0.1449...","[3.617496393086131, 0.6403988641975492, 0.1449...",11309.454215,106.345918,85.443627,10000
7,Original,MAE,0.00001,"(-0.0008095019450254399, 0.0034561029535432643...","[3.5271412239806383, 0.5402885473980458, 0.057...","[3.5271412239806383, 0.5402885473980458, 0.057...",12297.928348,110.896025,88.983460,10000
8,Original,MAE,0.00001,"(0.0, 0.0, 0.0)","[3.5275408809873916, 0.5369946172840103, 0.044...","[3.5275408809873916, 0.5369946172840103, 0.044...",12301.337809,110.911396,89.000437,10000
9,Standardized,MAE,0.00100,"(0.1, 0.1, 0.1)","[6.882833103328159, 1.3659667823491513, 3.7234...","[0.11964077921483865, 0.16042554786508506, 2.8...",82820.985065,287.786353,210.517090,10000
